In [1]:
%matplotlib widget
# %matplotlib inline

In [2]:
import json
import math
import subprocess
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from scipy.spatial import Delaunay
from matplotlib.pyplot import figure
from shapely.geometry import mapping, shape, MultiPoint, Point, GeometryCollection
from shapely.ops import triangulate
import pyproj
from matplotlib import colors
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
from scipy.spatial import SphericalVoronoi, geometric_slerp
from mpl_toolkits.mplot3d import proj3d
from descartes import PolygonPatch
from cartopy import crs as ccrs


ModuleNotFoundError: No module named 'cartopy'

In [ ]:
# Info: 
# GeoJson: [lng, lat]
# x y ~ lng lat

In [ ]:
# Load geojson
# Define locations
filename = "cities_pop_10000000.geojson"
    
with open(filename) as f:
    gj = json.load(f)
    points = MultiPoint([
        (feat['geometry']['coordinates'][0], feat['geometry']['coordinates'][1], float(feat['properties'].get('ele', '0')))
        for feat in gj['features']
    ])
    gs_points = gpd.GeoSeries([
        Point(feat['geometry']['coordinates'][0], feat['geometry']['coordinates'][1])
        for feat in gj['features']
    ])
    
print(points[0])
print("Loaded " + str(len(gs_points)) + " city coordinates")

In [ ]:
R = 6378137.0

In [ ]:
def get_haversine_distance(p1, p2):
    R = 6373.0
    lat1 = math.radians(p1.y)
    lon1 = math.radians(p1.x)
    lat2 = math.radians(p2.y)
    lon2 = math.radians(p2.x)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    # kilometer     
    return distance


def get_lowest_distance_in_poly(polygon, mode="normal", measure="min"):
    pc = polygon.centroid
    distances = []
    result = 0;

    if mode == "hausdorf":
        # Not working yet
        result = polygon.hausdorff_distance(pc)
    elif mode == "haversine":        
        for coords in np.array(polygon.boundary.coords):
            p1 = Point(coords)
            distances.append(get_haversine_distance(p1, pc))               
    else:
        for coords in np.array(polygon.boundary.coords):
            p1 = Point(coords)
            distances.append(p1.distance(pc))
                
    if measure == "min":
        return np.sort(distances)[0]
    elif measure == "avg":
        return sum(distances) / len(distances)
    else:
        return 0


def get_farthest_point(points, mode="delaunay", show=True):
    # triangulate geo_points
    triangles = triangulate(points)
    centroid_distances = []

    for idx, poly in enumerate(triangles):
        # plot triangles and centroids
        centroid_distances.append([idx, get_lowest_centroid_distance(poly, mode="normal")])

        # plot triangles and centroids
        if show:
            x,y = poly.exterior.xy
            plt.plot(poly.centroid.x, poly.centroid.y, 'ro')
            plt.plot(x,y)

    highest_distance_idx = sorted(centroid_distances, key=lambda x: x[1], reverse=True)[0]
    highest_distance_centroid = triangles[highest_distance_idx[0]].centroid

    if show:
        plt.plot(highest_distance_centroid.x, highest_distance_centroid.y, 'bo')
    
    return highest_distance_centroid

def gps_to_ecef(lng, lat, alt):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    
    x, y, z = pyproj.transform(lla, ecef, lng, lat, alt, radians=False)
    
    return [x, y, z]

def gps_to_ecef_custom(lng, lat, alt):
    rad_lat = lat * (math.pi / 180.0)
    rad_lng = lng * (math.pi / 180.0)

    a = R
    finv = 298.257223563
    f = 1 / finv
    e2 = 1 - (1 - f) * (1 - f)
    v = a / math.sqrt(1 - e2 * math.sin(rad_lat) * math.sin(rad_lat))

    x = (v + alt) * math.cos(rad_lat) * math.cos(rad_lng)
    y = (v + alt) * math.cos(rad_lat) * math.sin(rad_lng)
    z = (v * (1 - e2) + alt) * math.sin(rad_lat)

    return [x, y, 6378137.0]

def lnglat_2_xyz(lng, lat, alt):
    radius = R
    x = radius * math.cos(lat) * math.cos(lng)
    y = radius * math.cos(lat) * math.sin(lng)
    z = radius * math.sin(lat)
    
    return [x, y, z]

def get_geo_delaunay_data(dataset = 'cities_pop_3000000.geojson'):
    process = subprocess.Popen(["node", "geo_voronoi.js", dataset], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    for line in iter(process.stdout.readline, b''):
        print('Generated: ' + line.decode("utf-8").strip())

        datafile = line.decode("utf-8").strip()

        with open(datafile) as f:
            gj = json.load(f)

            return GeometryCollection([shape(feature["geometry"]) for feature in gj['features']])


In [ ]:
# set input data
sphere_points = np.array([gps_to_ecef_custom(pt.x, pt.y, pt.z) for pt in points])
sphere_points = np.array([lnglat_2_xyz(pt.x, pt.y, pt.z) for pt in points])

center = np.array([0, 0, 0])

# calculate spherical Voronoi diagram
sv = SphericalVoronoi(sphere_points, R, center)

# sort vertices (optional, helpful for plotting)
sv.sort_vertices_of_regions()

# generate plot
fig_3d = plt.figure()
ax_3d = fig_3d.add_subplot(111, projection='3d')

# plot the unit sphere for reference (optional)
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax_3d.plot_surface(x, y, z, color='y', alpha=0.1)

# plot generator points
ax_3d.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], c='b')

# plot Voronoi vertices
# ax_3d.scatter(sv.vertices[:, 0], sv.vertices[:, 1], sv.vertices[:, 2], c='g')

# indicate Voronoi regions (as Euclidean polygons)
for idx, region in enumerate(sv.regions):
    random_color = colors.rgb2hex(np.random.rand(3))
    polygon = Poly3DCollection([sv.vertices[region]], alpha=0.8)
    polygon.set_color(random_color)
    ax_3d.add_collection3d(polygon)
    

fig_3d.set_size_inches(8, 8)
plt.show()

In [ ]:
url = gpd.datasets.get_path('naturalearth_lowres')
df = gpd.read_file(url)
crs_projection = "EPSG:4326"

fig, ax = plt.subplots(figsize=(20,10))
ax.set_aspect('equal')

df.plot(ax=ax, color='white', edgecolor='black')

geo_delaunay_data = get_geo_delaunay_data(filename)
geo_delaunay_gs = gpd.GeoSeries(data=[poly for poly in geo_delaunay_data], crs=crs_projection)
geo_delaunay_gs = geo_delaunay_gs.to_crs(crs_projection)

geo_delaunay_gdf = gpd.GeoDataFrame(geometry=geo_delaunay_gs, crs=crs_projection)
# geo_delaunay_gdf = geo_delaunay_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')

for poly in geo_delaunay_data:
    x,y = poly.exterior.xy
    plt.plot(poly.centroid.x, poly.centroid.y, 'ro')
    plt.plot(x,y)



In [ ]:
# Define the CartoPy CRS object.
crs = ccrs.AzimuthalEquidistant()

# This can be converted into a `proj4` string/dict compatible with GeoPandas
crs_proj4 = crs.proj4_init
df_ae = df.to_crs(crs_proj4)
geo_delaunay_gdf_ae = geo_delaunay_gdf.to_crs(crs_proj4)

# Here's what the plot looks like in GeoPandas
df_ae.plot()

In [ ]:
crs_new = ccrs.Stereographic()
new_geometries = [crs_new.project_geometry(ii, src_crs=crs)
                  for ii in df_ae['geometry'].values]

new_geo_delaunay_data = [crs_new.project_geometry(ii, src_crs=crs)
                  for ii in geo_delaunay_gdf['geometry'].values]

fig, ax = plt.subplots(subplot_kw={'projection': crs_new})
ax.add_geometries(new_geometries, crs=crs_new)
ax.add_geometries(new_geo_delaunay_data, crs=crs_new)